In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
import sampling as sp
reload(fd)
reload(sd)
reload(di)
reload(resultMd)
reload(sp)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Sampling in Python

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

predictors = fd.getPredictors(dfTrx)
TEST_SIZE = 0.20 # test size using_train_test_split
RANDOM_STATE = 0
x_train0, x_test, y_train0, y_test = train_test_split(dfTrx[predictors], dfTrx['Class'], test_size = TEST_SIZE, 
                                                        stratify= dfTrx['Class'],
                                                        random_state = RANDOM_STATE)

#scaler=StandardScaler()
#x_train= scaler.fit_transform(x_train0)
#x_test= scaler.transform(x_test)
x_train= x_train0.copy()
x_test = x_test.copy()



# Over sampling
* randomOverSampling
* smoteOverSampling

# Under sampling
* randomUnderSampling
* NearMiss
* EditedNearestNeighboursUnderSampling (very slow)
* NeighbourhoodCleaningRuleUnderSampling
* tomekLinks (very slow 33 minutes)

# Combined approach
* Combined random over and underSampling


# Over sampling

In [ ]:
x_train, y_train = sp.randomOverSampling(x_train0, y_train0,rateOverSampling=5)
sp.draw2DPCAScaterPlot(x_train0,y_train0)
sp.draw2DPCAScaterPlot(x_train,y_train)

In [ ]:

x_train, y_train = sp.smoteOverSampling(x_train0, y_train0,rateOverSampling=5)

sp.draw2DPCAScaterPlot(x_train0,y_train0)
sp.draw2DPCAScaterPlot(x_train,y_train)


# Under sampling

In [ ]:
x_train, y_train = sp.randomUnderSampling(x_train0, y_train0,rate=1/100)
sp.draw2DPCAScaterPlot(x_train0,y_train0)
sp.draw2DPCAScaterPlot(x_train,y_train)

In [ ]:
x_train, y_train = sp.nearMissV1UnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlot(x_train0,y_train0)
sp.draw2DPCAScaterPlot(x_train,y_train)


In [ ]:
x_train, y_train = sp.nearMissV2UnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlot(x_train0,y_train0)
sp.draw2DPCAScaterPlot(x_train,y_train)

In [ ]:
x_train, y_train = sp.nearMissV3UnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlot(x_train0,y_train0)
sp.draw2DPCAScaterPlot(x_train,y_train)

In [ ]:


x_train, y_train = sp.tomekLinksUnderSampling(x_train0, y_train0)

sp.draw2DPCAScaterPlot(x_train0,y_train0)
sp.draw2DPCAScaterPlot(x_train,y_train)

In [ ]:


x_train, y_train = sp.neighbourhoodCleaningRuleUnderSampling(x_train0, y_train0,rate=1/100)

sp.draw2DPCAScaterPlot(x_train0,y_train0)
sp.draw2DPCAScaterPlot(x_train,y_train)

# combined approach

In [ ]:
x_train, y_train = sp.randomOverUnderSampling(x_train0, y_train0)
sp.draw2DPCAScaterPlot(x_train0,y_train0)
sp.draw2DPCAScaterPlot(x_train,y_train)

# combined version  EditedNearestNeighbours RandomUnderSampler

In [ ]:

from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from datetime import datetime
from imblearn.under_sampling import OneSidedSelection
from sklearn.model_selection import train_test_split

then= datetime.now()
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200)
print("1-x_train.shape",x_train0.shape)
now = datetime.now()
print(datetime.now())
x_train, y_train = undersample.fit_resample(x_train0, y_train0)
print(datetime.now())
print("2-x_train.shape",x_train.shape)
duration= now - then
duration_in_s = duration.total_seconds()
print("1-Duration ",duration_in_s)

modelClf = KNeighborsClassifier()
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("2-Duration ",duration_in_s)

#resultMd.update_time_response_result('05-sklearn.neighbors','KNeighborsClassifier','2-DownSampling', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
#resultMd.update_performance_test_result('05-sklearn.neighbors','KNeighborsClassifier','2-DownSampling', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test)

